In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.utils import shuffle

2025-02-23 21:07:08.181689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load data from file
data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

scores = ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
          'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']

# Extract relevant columns
x_train = data[scores].astype(np.float32)
y_train = data["Class/ASD"].values.astype(np.float32)

x_test = test_data[scores].astype(np.float32)
y_test = (test_data["austim"].values == "yes").astype(np.float32)

In [5]:
class ASDClassifier(nn.Module): 

    def __init__(self, input_dim: int): 
        super().__init__()

        # Layers
        self.input_layer = nn.Linear(input_dim, 64)
        self.hidden1 = nn.Linear(64, 64)
        self.hidden2 = nn.Linear(64, 64)
        self.hidden3 = nn.Linear(64, 64)
        self.output_layer = nn.Linear(64, 1)

        # Relevant activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x): 
        x_1 = self.relu(self.input_layer(x))
        x_2 = self.relu(self.hidden1(x_1))
        x_3 = self.relu(self.hidden2(x_2))
        x_4 = self.relu(self.hidden2(x_3))
        x_5 = self.sigmoid(self.output_layer(x_4))
        return x_5
    

# Instantiate the model
torch_nn_model = ASDClassifier(x_train.shape[1])

# Define the loss and optimizer
loss_fn = nn.BCELoss()
optimizer = optim.Adam(torch_nn_model.parameters(), lr=1e-4)

# Training parameters
batch_size = 32
epochs = 100

# Training loop
for epoch in range(epochs):

    shuffled_x, shuffled_y = shuffle(x_train, y_train)

    # Convert dataframe to tensors
    x_train_as_tensor = torch.tensor(shuffled_x.values)
    y_train_as_tensor = torch.tensor(shuffled_y)

    for batch in range(0, x_train.shape[0], batch_size):

        # Batching
        xbatch = x_train_as_tensor[batch:batch+batch_size]
        ybatch_true = y_train_as_tensor[batch:batch+batch_size].unsqueeze(1)

        # Model predictions
        ybatch_pred = torch_nn_model(xbatch)

        # Loss
        loss = loss_fn(ybatch_pred, ybatch_true)

        # Zero gradient
        optimizer.zero_grad()

        # Backpropagation
        loss.backward()

        # Update
        optimizer.step()

    # Report each epoch
    print(f"Finished training epoch {epoch + 1}; loss {loss}", flush=True)

Finished training epoch 1; loss 0.7084486484527588
Finished training epoch 2; loss 0.6929503679275513
Finished training epoch 3; loss 0.6815835237503052
Finished training epoch 4; loss 0.6927056908607483
Finished training epoch 5; loss 0.6453129053115845
Finished training epoch 6; loss 0.6156519651412964
Finished training epoch 7; loss 0.5911471843719482
Finished training epoch 8; loss 0.4844778776168823
Finished training epoch 9; loss 0.3853606879711151
Finished training epoch 10; loss 0.49234044551849365
Finished training epoch 11; loss 0.4407377541065216
Finished training epoch 12; loss 0.4629581570625305
Finished training epoch 13; loss 0.483237624168396
Finished training epoch 14; loss 0.4470459520816803
Finished training epoch 15; loss 0.3564450740814209
Finished training epoch 16; loss 0.2711825966835022
Finished training epoch 17; loss 0.23510044813156128
Finished training epoch 18; loss 0.28066083788871765
Finished training epoch 19; loss 0.43213728070259094
Finished training 

In [4]:
tf_nn_model = tf.keras.Sequential([
    tf.keras.Input(shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# Spe fy the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compiling the model
tf_nn_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Training the model
history = tf_nn_model.fit(x=x_train, y=y_train, batch_size=32, epochs=100)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7841 - loss: 0.6193
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8195 - loss: 0.4146
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8339 - loss: 0.3589
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8459 - loss: 0.3122
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8456 - loss: 0.3022 
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8619 - loss: 0.2891 
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8383 - loss: 0.3073 
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8690 - loss: 0.2633 
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8644 - loss: 0.2866 
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8809 - loss: 0.2491 
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8887 - loss: 0.2527 
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - ac